<a href="https://colab.research.google.com/github/donghuna/PromptGenerate/blob/main/Evaluation-agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
from types import FunctionType
import re
from tqdm import tqdm

In [ ]:
humanEval_dataset = load_dataset("donghuna/generated_code-humanEval_prompt")
humanEval_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/418 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/164 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['task_id', 'prompt', 'solution', 'test', 'entry_point'],
        num_rows: 164
    })
})

In [ ]:
def execute_code(code, entry_point):


    try:
        exec(code, globals())

        check_func = globals()['check']

        candidate_func = globals().get(entry_point)
        if not isinstance(candidate_func, FunctionType):
            raise ValueError(f"{entry_point} is not a function")

        # check 함수로 검증 실행
        check_func(candidate_func)
        return True
    # except AssertionError:
    #     print("Test failed: The generated solution did not pass the tests.")
    #     return False
    # except KeyError as e:
    #     print(f"Error: Missing required function or variable - {e}")
    #     return False
    # except ValueError as ve:
    #     print(f"Error: {ve}")
    #     return False
    # except NameError as ne:
    #     print(f"Error: NameError - {ne}")
    #     return False
    except Exception as e:
        return str(e)

In [ ]:
passed = 0
syntaxError = 0
AssertionError = 0
KeyError = 0
ValueError = 0
NameError = 0

for row in tqdm(humanEval_dataset['train'], desc="HumanEval dataset", unit="sample"):
    task_id = row['task_id']
    prompt = row['prompt']
    solution = row['solution']
    test_code = row['test']
    entry_point = row['entry_point']

    combined_code = solution
    combined_code += '\n' + test_code

    result = execute_code(combined_code, entry_point)

    if result == True:
        passed += 1
    else:
        print(f"Failed: {task_id} : {result}")

print(f"Passed: {passed}/{len(humanEval_dataset['train'])}")


HumanEval dataset: 100%|██████████| 164/164 [00:00<00:00, 1825.64sample/s]

Failed: HumanEval/1 : 
Failed: HumanEval/2 : unterminated triple-quoted string literal (detected at line 71) (<string>, line 58)
Failed: HumanEval/3 : 
Failed: HumanEval/4 : expected an indented block after 'if' statement on line 16 (<string>, line 19)
Failed: HumanEval/5 : 
Failed: HumanEval/6 : 
Failed: HumanEval/7 : unterminated string literal (detected at line 14) (<string>, line 14)
Failed: HumanEval/10 : local variable 'i' referenced before assignment
Failed: HumanEval/11 : expected an indented block after 'if' statement on line 14 (<string>, line 18)
Failed: HumanEval/12 : expected an indented block after 'if' statement on line 23 (<string>, line 26)
Failed: HumanEval/14 : 
Failed: HumanEval/15 : expected an indented block after 'if' statement on line 13 (<string>, line 16)
Failed: HumanEval/17 : 
Failed: HumanEval/18 : string index out of range
Failed: HumanEval/19 : '(' was never closed (<string>, line 17)
Failed: HumanEval/20 : 
Failed: HumanEval/21 : '(' was never closed (<s